In [2]:
#Imports
import pandas as pd
import pyodbc
import sqlalchemy
from sqlalchemy import create_engine
import urllib
import numpy as np

In [3]:
# Watermark
print('Nathan Young\nJunior Data Analyst\nCenter for the Study of Free Enterprise')
%load_ext watermark
%watermark -a "Western Carolina University" -u -d -p pandas

Nathan Young
Junior Data Analyst
Center for the Study of Free Enterprise
Western Carolina University 
last updated: 2020-02-22 

pandas 1.0.1


In [4]:
# Create Backups
#df_backup = pd.read_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt')
#df_backup.to_csv('./Backups/STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP.txt')

In [12]:
#Load Land data
df_mlp = pd.read_csv('http://files.zillowstatic.com/research/public/County/County_MedianListingPricePerSqft_AllHomes.csv',
                     encoding='ISO-8859-1')
#Display table to ensure data loaded correctly
df_mlp.head()

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
0,Los Angeles County,CA,Los Angeles-Long Beach-Anaheim,6,37,1,268.016677,262.582145,268.548443,268.606121,...,432.432432,435.166431,436.875000,439.276486,441.806331,441.269841,444.829946,448.379052,449.479941,448.828184
1,Cook County,IL,Chicago-Naperville-Elgin,17,31,2,188.180379,189.909091,190.106431,189.677419,...,202.127660,205.553108,203.600000,202.721262,200.000000,197.415650,195.833333,193.627451,191.858605,191.083823
2,Harris County,TX,Houston-The Woodlands-Sugar Land,48,201,3,75.410564,76.256732,76.475902,76.731553,...,120.912149,122.250291,122.107192,122.064428,121.498946,121.393385,121.811953,121.871875,122.312824,122.575298
3,Maricopa County,AZ,Phoenix-Mesa-Scottsdale,4,13,4,99.009568,97.445125,96.463023,95.029240,...,166.111851,167.022962,166.666667,166.590806,166.790490,167.290886,168.903286,170.751494,171.685254,172.116503
4,San Diego County,CA,San Diego-Carlsbad,6,73,5,246.596629,249.456682,250.885478,251.233096,...,377.911498,381.207028,382.097052,383.771930,382.409178,381.943403,383.008357,382.146564,386.654883,388.864043


In [14]:
#Filter data to NC

filter1 = df_mlp['State'] == "NC"
df_mlp_nc = df_mlp[filter1]

#Check to ensure filter worked
df_mlp_nc.head(5)

,RegionName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12
48,Mecklenburg County,NC,Charlotte-Concord-Gastonia,37,119,49,102.377133,103.203862,102.811589,102.969697,...,149.492366,150.998306,149.258536,149.595480,149.501343,148.867079,150.277557,150.740132,150.787154,151.346664
55,Wake County,NC,Raleigh,37,183,56,116.388922,115.765314,115.622242,115.518882,...,147.058824,148.395515,149.074923,149.130075,149.434540,149.515561,149.798967,149.979135,150.178827,150.812065
134,Guilford County,NC,Greensboro-High Point,37,81,135,92.495637,92.846208,93.487395,92.374452,...,108.420139,109.375000,108.813928,108.673913,108.695652,107.889049,108.448001,109.170307,108.961960,109.363514
181,Forsyth County,NC,Winston-Salem,37,67,182,97.085492,97.529259,98.065959,97.793190,...,110.135743,112.705357,113.955000,115.136928,114.942529,114.921227,114.924446,115.913918,117.317193,119.921773
196,Cumberland County,NC,Fayetteville,37,51,197,99.226804,99.210582,100.122926,100.474595,...,103.452382,106.007230,105.721393,107.284768,106.483126,106.114650,104.716981,103.940747,103.576751,102.849112


In [ ]:
#View data types of dataframe
df_mlp_nc.dtypes

In [ ]:
#Change MunicipalCodeFIPS dtype to add leading 0's
df_mlp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mlp_nc['MunicipalCodeFIPS'].astype(str)
df_mlp_nc.dtypes

In [ ]:
#Add leading 0's and check to ensure they were added
df_mlp_nc.loc[ :, 'MunicipalCodeFIPS'] = df_mlp_nc['MunicipalCodeFIPS'].str.zfill(3)
df_mlp_nc.head(5)

In [ ]:
# Set Index to Region Name
df_mlp_nc.set_index(df_mlp_nc['RegionName'], inplace = True)
df_mlp_nc

In [ ]:
# Drop Region Name column
df_mlp_nc.drop('RegionName', axis = 1, inplace = True)
df_mlp_nc

In [ ]:
#Save to csv file for export in Excel
df_mlp_nc.to_csv('./Updates/STG_ZLLW_County_MedianListingPrice_AllHomes.txt', sep ='\t')

In [ ]:
#Reset Index for upload to database
df_mlp_nc = df_mlp_nc.reset_index()    

In [ ]:
#Fill NaN values for upload to database
df_mlp_nc['Metro'] = df_mlp_nc['Metro'].replace(np.nan,'', regex=True)

column_list = df_mlp_nc.columns.values
for i in column_list:
    df_mlp_nc.loc[df_mlp_nc[i].isnull(),i]=0

In [ ]:
#Connect to database and create cursor
con = pyodbc.connect('Driver={SQL Server};'
                      'Server=TITANIUM-BOOK;'
                      'Database=DataDashboard;'
                      'Trusted_Connection=yes;',
                    autocommit=True)

c = con.cursor()

In [ ]:
#Drop old backup table
c.execute('drop table STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP')

In [ ]:
#Create new backup
c.execute('''sp_rename 'dbo.STG_ZLLW_County_MedianListingPrice_AllHomes','STG_ZLLW_County_MedianListingPrice_AllHomes_BACKUP';''')

In [ ]:
c.execute('''USE [DataDashboard]

SET ANSI_NULLS ON

SET QUOTED_IDENTIFIER ON

CREATE TABLE [dbo].[STG_ZLLW_County_MedianListingPrice_AllHomes](
	[RegionName] [varchar](40) NULL,
	[State] [varchar](2) NULL,
	[Metro] [varchar](40) NULL,
	[StateCodeFIPS] [varchar](2) NULL,
	[MunicipalCodeFIPS] [varchar](3) NULL,
	[SizeRank] [smallint] NULL,
	[2010-01] [float] NULL,
	[2010-02] [float] NULL,
	[2010-03] [float] NULL,
	[2010-04] [float] NULL,
	[2010-05] [float] NULL,
	[2010-06] [float] NULL,
	[2010-07] [float] NULL,
	[2010-08] [float] NULL,
	[2010-09] [float] NULL,
	[2010-10] [float] NULL,
	[2010-11] [float] NULL,
	[2010-12] [float] NULL,
	[2011-01] [float] NULL,
	[2011-02] [float] NULL,
	[2011-03] [float] NULL,
	[2011-04] [float] NULL,
	[2011-05] [float] NULL,
	[2011-06] [float] NULL,
	[2011-07] [float] NULL,
	[2011-08] [float] NULL,
	[2011-09] [float] NULL,
	[2011-10] [float] NULL,
	[2011-11] [float] NULL,
	[2011-12] [float] NULL,
	[2012-01] [float] NULL,
	[2012-02] [float] NULL,
	[2012-03] [float] NULL,
	[2012-04] [float] NULL,
	[2012-05] [float] NULL,
	[2012-06] [float] NULL,
	[2012-07] [float] NULL,
	[2012-08] [float] NULL,
	[2012-09] [float] NULL,
	[2012-10] [float] NULL,
	[2012-11] [float] NULL,
	[2012-12] [float] NULL,
	[2013-01] [float] NULL,
	[2013-02] [float] NULL,
	[2013-03] [float] NULL,
	[2013-04] [float] NULL,
	[2013-05] [float] NULL,
	[2013-06] [float] NULL,
	[2013-07] [float] NULL,
	[2013-08] [float] NULL,
	[2013-09] [float] NULL,
	[2013-10] [float] NULL,
	[2013-11] [float] NULL,
	[2013-12] [float] NULL,
	[2014-01] [float] NULL,
	[2014-02] [float] NULL,
	[2014-03] [float] NULL,
	[2014-04] [float] NULL,
	[2014-05] [float] NULL,
	[2014-06] [float] NULL,
	[2014-07] [float] NULL,
	[2014-08] [float] NULL,
	[2014-09] [float] NULL,
	[2014-10] [float] NULL,
	[2014-11] [float] NULL,
	[2014-12] [float] NULL,
	[2015-01] [float] NULL,
	[2015-02] [float] NULL,
	[2015-03] [float] NULL,
	[2015-04] [float] NULL,
	[2015-05] [float] NULL,
	[2015-06] [float] NULL,
	[2015-07] [float] NULL,
	[2015-08] [float] NULL,
	[2015-09] [float] NULL,
	[2015-10] [float] NULL,
	[2015-11] [float] NULL,
	[2015-12] [float] NULL,
	[2016-01] [float] NULL,
	[2016-02] [float] NULL,
	[2016-03] [float] NULL,
	[2016-04] [float] NULL,
	[2016-05] [float] NULL,
	[2016-06] [float] NULL,
	[2016-07] [float] NULL,
	[2016-08] [float] NULL,
	[2016-09] [float] NULL,
	[2016-10] [float] NULL,
	[2016-11] [float] NULL,
	[2016-12] [float] NULL,
	[2017-01] [float] NULL,
	[2017-02] [float] NULL,
	[2017-03] [float] NULL,
	[2017-04] [float] NULL,
	[2017-05] [float] NULL,
	[2017-06] [float] NULL,
	[2017-07] [float] NULL,
	[2017-08] [float] NULL,
	[2017-09] [float] NULL,
	[2017-10] [float] NULL,
	[2017-11] [float] NULL,
	[2017-12] [float] NULL,
	[2018-01] [float] NULL,
	[2018-02] [float] NULL,
	[2018-03] [float] NULL,
	[2018-04] [float] NULL,
	[2018-05] [float] NULL,
	[2018-06] [float] NULL,
	[2018-07] [float] NULL,
	[2018-08] [float] NULL,
	[2018-09] [float] NULL,
	[2018-10] [float] NULL,
	[2018-11] [float] NULL,
	[2018-12] [float] NULL,
	[2019-01] [float] NULL,
	[2019-02] [float] NULL,
	[2019-03] [float] NULL,
	[2019-04] [float] NULL,
	[2019-05] [float] NULL,
	[2019-06] [float] NULL,
	[2019-07] [float] NULL,
	[2019-08] [float] NULL,
	[2019-09] [float] NULL,
	[2019-10] [float] NULL,
	[2019-11] [float] NULL,
	[2019-12] [float] NULL,
    [2020-01] [float] NULL,
    [2020-02] [float] NULL,
    [2020-03] [float] NULL,
    [2020-04] [float] NULL,
    [2020-05] [float] NULL,
    [2020-06] [float] NULL,
    [2020-07] [float] NULL,
    [2020-08] [float] NULL,
    [2020-09] [float] NULL,
    [2020-10] [float] NULL,
    [2020-11] [float] NULL,
    [2020-12] [float] NULL,
    [2021-01] [float] NULL,
    [2021-02] [float] NULL,
    [2021-03] [float] NULL,
    [2021-04] [float] NULL,
    [2021-05] [float] NULL,
    [2021-06] [float] NULL
    [2021-07] [float] NULL,
    [2021-08] [float] NULL,
    [2021-09] [float] NULL,
    [2021-10] [float] NULL,
    [2021-11] [float] NULL,
    [2021-12] [float] NULL,
    [2022-01] [float] NULL,
    [2022-02] [float] NULL,
    [2022-03] [float] NULL,
    [2022-04] [float] NULL,
    [2022-05] [float] NULL,
    [2022-06] [float] NULL,
    [2022-07] [float] NULL,
    [2022-08] [float] NULL,
    [2022-09] [float] NULL,
    [2022-10] [float] NULL,
    [2022-11] [float] NULL,
    [2022-12] [float] NULL
) ON [PRIMARY]''')

In [ ]:
params = urllib.parse.quote_plus(r'Driver={SQL Server};' 
                                 r'Server=TITANIUM-BOOK;'
                                 r'Database=DataDashboard;'
                                 r'Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

#warning: discard old table if exists
df_mlp_nc.to_sql('STG_ZLLW_County_MedianListingPrice_AllHomes', con=engine, if_exists='replace', index=False)